In [ ]:
# Example of pKa

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)
show_progress = True

## Define a molecule you like

In [ ]:
smiles = "CCOC(=O)C1=C(C)N=C(C)/C(=C(\O)OCC)C1C"  # CHEMBL3189958
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

## Generate tautomers of the molecule

Generate tautomer configuration based on RDKit enumeration

Reference
 - http://rdkit.blogspot.com/2020/01/trying-out-new-tautomer.html


In [ ]:
enumerator = rdMolStandardize.TautomerEnumerator()

In [ ]:
tautomers = enumerator.Enumerate(molobj)
tautomers = list(tautomers)

In [ ]:
MolsToGridImage(tautomers)

## Use xTB to select which tautomer is the most stable

- Expand the configurations into 3D conformers
- Calculate the relative energy in water
- Optimize the molecule a bit (crude optimization critiera)


Probably a single point (no optimization) would be enough. However, sometimes FF conformer generation can give unnatural high energies, and so we optimize to avoid that.


Reference
- https://xtb-docs.readthedocs.io/en/latest/optimization.html


In [ ]:
xtb = ppqm.xtb.XtbCalculator(
    scr="_tmp_directory_", n_cores=2, cmd="xtb", show_progress=show_progress
)

In [ ]:
xtb_options = {
    "gfn": 2,
    "alpb": "water",
    "opt": "crude", 
}

In [ ]:
# Generate conformers
for i, molobj in enumerate(tautomers):
    molobj = ppqm.tasks.generate_conformers(molobj, n_conformers=5)
    tautomers[i] = molobj
    
    id(molobj)
    
    print(f"Tautomer {i} has {molobj.GetNumConformers()} conformers")

In [ ]:
%%time

tautomer_energies = np.zeros(len(tautomers))

for i, molobj in enumerate(tautomers):

    results = xtb.calculate(molobj, xtb_options)
    
    energies = [result["scc_energy"] for result in results]
    
    min_energy = np.min(energies)
    min_energy *= ppqm.units.hartree_to_kcalmol
    
    tautomer_energies[i] = min_energy

In [ ]:
# We are only interested in relative energies
tautomer_energies -= np.min(tautomer_energies)

In [ ]:
tautomer_energies

In [ ]:
_ = plt.plot(tautomer_energies, "kx")

In [ ]:
print(f"Best tautomer based on xTB energy is Tautomer #{np.argmin(tautomer_energies)}")

## Error in energy

As we with anything fast, xTB energies comes with a cost. There is an error associated with the energy, and to an extent, also the sampled conformer space.  

So let's limit how accurate we think the energy is and pick all the tautomers lower than that.


In [ ]:
energy_cutoff = 7.0  # kcal/mol

In [ ]:
n_tautomers = len(tautomer_energies)
_ = plt.plot(tautomer_energies, "kx")
_ = plt.plot(range(n_tautomers), [energy_cutoff]*n_tautomers, "r-")

In [ ]:
stable_indices, = np.where(tautomer_energies < energy_cutoff)
stable_tautomers = [tautomers[i] for i in stable_indices]
stable_tautomers = [ppqm.chembridge.copy_molobj(x) for x in stable_tautomers]
stable_tautomers = [Chem.RemoveHs(x) for x in stable_tautomers]

## Results

So assuming that RDKit generates all relevant tautomers, and xTB energy is good to within a certain threshold, these are the tautomers that are abundant in water;


In [ ]:
MolsToGridImage(stable_tautomers)